In [1]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing


# desired_width = 500
# pd.set_option("display.width", desired_width)
# np.set_printoptions(linewidth=desired_width)
# pd.set_option("display.max_columns", 30)
import pprint
pd.set_option('display.max_columns', None)
# Ustawienie maksymalnej ilości wyświetlanych wierszy
pd.set_option('display.max_rows', None)


In [2]:
classes = pd.DataFrame(
    data={
        "classes": [
            "pump",
            "tank",
            "seepage_boxes",
            "diameter_increase",
            "diameter_reduction",
            "slope_increase",
            "slope_reduction",
            "depth_increase",
            "depth_reduction",
            "valid",
        ]
    }
)
classes

,classes
0,pump
1,tank
2,seepage_boxes
3,diameter_increase
4,diameter_reduction
5,slope_increase
6,slope_reduction
7,depth_increase
8,depth_reduction
9,valid


In [3]:
import sys
print(sys.path)

['c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\stormwater_analysis\\data\\recomendations\\dataset', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv', '', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\win32', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\Pythonwin']


In [4]:
import os
os.chdir('C:\\Users\\Dell\\Documents\\Git\\stormwater-analysis')

from stormwater_analysis.inp_manage.inp import SwmmModel

In [5]:
INP_FILE = R"C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\recomendations\dataset\diameter\02_diameter.inp"

with pyswmm.Simulation(INP_FILE) as sim:
    for _ in sim:
        pass

from stormwater_analysis.inp_manage.inp import SwmmModel
from stormwater_analysis.data.feature_engineering import feature_engineering

model = swmmio.Model(INP_FILE, include_rpt=True)

with pyswmm.Simulation(model.inp.path) as sim:
    for _ in sim:
        pass

conduits_data, nodes_data, subcatchments_data = feature_engineering(model)
swmm_model = SwmmModel(model, conduits_data, nodes_data, subcatchments_data)

1/1 [==============================] - 0s 137ms/step


In [6]:
print(swmm_model.conduits_data.conduits.iloc[:, 18:])

                    Tag  Filling  ValMaxFill  ValMaxV  ValMinV  ValMaxSlope  ValMinSlope  InletMaxDepth  OutletMaxDepth  SlopePerMile  InletGroundElevation  OutletGroundElevation  InletGroundCover  OutletGroundCover  ValDepth  ValCoverage
Name                                                                                                                                                                                                                                          
C3                valid    0.594           1        1        1            1            1           2.71            2.40           1.8                149.32                 148.92              1.81             294.54         1            1
C4    diameter_increase    1.100           1        1        1            1            1           2.40            2.08           6.4                148.92                 148.28              0.40             292.48         1            0
C2    diameter_increase    0.380           1

In [7]:
print(swmm_model.conduits_data.conduits.iloc[:, 15:35])
# self.conduits["ValCoverage"] = (
#     (self.conduits.InletGroundElevation <= (self.conduits.InletNodeInvert - self.frost_zone))
#     & (self.conduits.OutletGroundElevation <= (self.conduits.OutletNodeInvert - self.frost_zone))
# ).astype(int)

# UpstreamInvert  - to jest rzędna dna kanału
# InletMaxDepth  -  to jest całkowite zagłębienie kanału - czyli od powierzchni terenu do dna kanału
# InletGroundElevation - czyli pokrywa z gruntu powinna być obliczna
# a walidacja pokrywy zatem obliczona jako 
# InletGroundElevation  - 

      UpstreamInvert  DownstreamInvert  SlopeFtPerFt                Tag  Filling  ValMaxFill  ValMaxV  ValMinV  ValMaxSlope  ValMinSlope  InletMaxDepth  OutletMaxDepth  SlopePerMile  InletGroundElevation  OutletGroundElevation  InletGroundCover  OutletGroundCover  ValDepth  ValCoverage
Name                                                                                                                                                                                                                                                                                          
C3            146.61            146.52        0.0018              valid    0.594           1        1        1            1            1           2.71            2.40           1.8                149.32                 148.92              1.81             294.54         1            1
C4            146.52            146.20        0.0064  diameter_increase    1.100           1        1        1            1            1   

In [8]:
print(swmm_model.conduits_data.conduits)

     InletNode OutletNode  Length  Roughness  MaxFlow  Geom1     Type   MaxQ  MaxDay  MaxHr  MaxV  MaxQPerc  MaxDPerc  InletNodeInvert  OutletNodeInvert  ...  Filling  ValMaxFill  ValMaxV ValMinV  ValMaxSlope  ValMinSlope  InletMaxDepth  OutletMaxDepth  SlopePerMile  InletGroundElevation  OutletGroundElevation  InletGroundCover  OutletGroundCover  ValDepth  ValCoverage
Name                                                                                                                                                      ...                                                                                                                                                                                                                      
C3          J2         J3      50      0.010        0    0.9  CONDUIT  0.772       0  00:17  1.76      0.77      0.66           146.61            146.52  ...    0.594           1        1       1            1            1           2.71            2.40    

In [9]:
"pump",
"tank",
"seepage_boxes",
"diameter_increase",
"diameter_reduction",
"slope_increase",
"slope_reduction",
"depth_increase",
"depth_reduction",
"valid",

('valid',)